<a href="https://colab.research.google.com/github/Xixaus/Nastroje/blob/main/CD_extrakce%2Bvyhlazen%C3%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Pro spuštění skriptu je nutné se přihlásit do Google účtu.
2. Spusťte 1. buňku tlačítkem ▶ v levém horním rohu nebo kliknutím do buňky a stistnutím klávesové zkratky ctrl+enter aby se načetly knihovny a vytvořili složky.
3. Do složky Samples vložte soubory s vzorky a do složky blanks vložte soubory s blanky

    !!! Je nutné aby pH vzoku bylo v názvu v souboru v následujícím formátu: ... _ 7_05_pufr.txt, kdy tímto způsobem bude končit název souboru !!!

3. Spusťte 2. buňku pro vygenerování grafu a souboru
    
    !!! pokud !!!
4. Soubor data.txt vložte do Bestselu, pokud Vám bude hlásit chybu, zkopírujte všechny data ze souboru a vložte do okna v Bestselu

In [12]:
#@title Volby { run: "auto", vertical-output: true }

choice = "Absorbance" # @param ["Elipticita", "Absorbance", "Fluorescence"]

In [4]:
#@title Knihovny a vytvoření/smazání složek { run: "auto", vertical-output: true }

#načtení knihoven
import os
import sys
import shutil
import pandas as pd
from scipy.signal import savgol_filter
import plotly.graph_objects as go

#vytvoření potřebných složek pro data
if os.path.exists("/content/samples"):
    shutil.rmtree("/content/samples")
os.makedirs("/content/samples", exist_ok=True)

if os.path.exists("/content/blanks"):
    shutil.rmtree("/content/blanks")
os.makedirs("/content/blanks", exist_ok=True)

#smazání zbytečné složky
if os.path.exists("/content/sample_data"):
    shutil.rmtree("/content/sample_data")

In [14]:
#@title Výpočty { run: "auto", vertical-output: true }

#funkce pro extrakci dat ze souborů

def extract_data(folder_path, col):
    file_list = os.listdir(folder_path)
    df = pd.DataFrame() #vytvoření prázného dataframe

    for filename in file_list: #iterace každého souboru ve složce
        if filename.endswith(".txt"): #podmínka pro extrakci pouze z .txt souborů
            number = (filename.split("_")[-3].split(".")[0]) + "." + filename.split("_")[-2].split(".")[0] #pojmenování spektra, konkrétně pH, které je na konci ve formátu ..._7_05_pufr.txt
            data = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[0], header=None) #extrakce vlnových délek

            start = data[data[0] == "XYDATA"].index[0] + 1

            try:
                end = data[data == "##### Extended Information"].dropna(how='all').index.item()
            except:
                end = None

            axis = data.iloc[start:end, 0].values.flatten()
            spectrum = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[col], skiprows=start, nrows=len(axis), header=None)
            spectrum = spectrum.rename(columns={col: float(number)})#přejmenování spektra podle promněnné number
            spectrum.index = axis

            df = pd.concat([df, spectrum], axis=1) #spojení dataframe se spectrem

    df = df.sort_index(axis=1) #seřazení sepekter podle názvu (number)
    df = df.applymap(lambda x: float(x.replace(',', '.')))
    df.index = df.index.to_series().apply(lambda x: float(x.replace(',', '.')))
    return(df)

directory = {"Elipticita": 1, "Absorbance": 2,"Fluorescence": 3}

col = directory[choice]

samples = extract_data("/content/samples", col) #extrakce naměřených spekter
blanks = extract_data("/content/blanks", col) #extrakce blanků

spectra = samples - blanks
spectra_SG = samples - blanks #odečtení blanků od spekter


for column in spectra_SG.columns:
    spectra_SG[column] = savgol_filter(spectra_SG[column], window_length=7, polyorder=2, deriv=0) #vyhlazení pomocí savitzkého-golayových filtrů

fig1 = go.Figure() #vykreslení grafu

for column in spectra.columns:
    fig1.add_trace(go.Scatter(y=spectra[column], x=spectra.index, mode='lines', name=str(column)))

fig1.update_layout(xaxis_title="λ [nm]",
                   yaxis_title=choice)
fig1.update_layout(title="Spektra")
fig1.show()


display(spectra)


spectra.to_csv(f'spectra_{choice}.txt', sep='\t') #uložení spekter do .txt souboru
spectra_SG.to_csv(f'spectra_vyhlazena_{choice}.txt', sep='\t') #uložení spekter do .txt souboru

,-7.05,-6.05,-5.05,-4.05,-3.05
350.0,0.002480,0.003692,0.005261,0.005860,0.007278
349.0,0.002158,0.003746,0.005103,0.006065,0.007119
348.0,0.002202,0.003770,0.005379,0.006347,0.007640
347.0,0.002386,0.003956,0.005539,0.006705,0.008181
346.0,0.002138,0.003863,0.005613,0.006782,0.008454
...,...,...,...,...,...
204.0,0.489490,0.507920,0.526740,0.517780,0.553860
203.0,0.552450,0.567900,0.584630,0.569260,0.606800
202.0,0.624510,0.638680,0.653970,0.630980,0.668980
201.0,0.708290,0.721020,0.733860,0.703020,0.743120


In [13]:
fig2 = go.Figure() #vykreslení grafu

for column in spectra.columns:
    fig2.add_trace(go.Scatter(y=spectra_SG[column], x=spectra_SG.index, mode='lines', name=str(column)))

fig2.update_layout(xaxis_title="λ [nm]",
                   yaxis_title=choice)
fig2.update_layout(title="Vyhlazená spektra")
fig2.show()

spectra_SG.astype("float16")

display(spectra_SG)

,-7.05,-6.05,-5.05,-4.05,-3.05
350.0,0.066551,0.030308,0.046521,-0.018579,0.001490
349.0,0.039883,0.019413,0.020195,-0.016953,-0.001090
348.0,0.023691,0.012780,0.007357,-0.015845,-0.003541
347.0,0.017975,0.010410,0.008005,-0.015255,-0.005863
346.0,0.037021,0.008196,0.038892,-0.020703,-0.013235
...,...,...,...,...,...
204.0,0.519835,0.670761,0.640121,0.347429,0.843307
203.0,0.896698,0.790463,0.583353,0.491144,0.779031
202.0,0.912465,1.055224,0.724237,0.619118,0.645433
201.0,0.771231,1.379222,0.903437,0.752533,0.422782
